In [0]:
!pip install sklearn_crfsuite

In [0]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [0]:
pip install spacy && python -m spacy download pt

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [0]:
!pip install tweepy

In [0]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tweepy
import os
import re

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/dataset.ptbr_tweets.txt', sep="\t")

consumer_key = 'gPTv31AWDAcfT2Qt07tCVv2dy'
consumer_secret = 'BiYelgBy09zC7fsFEabsSkzyWWc0cuL2EXi6XNcrX4hKAnOCje'
access_token = '483135262-TGZaFpcMznLlYhNfC5ABwrK3urkh4FKLuHuw6Xlz'
access_token_secret = '9Jon32GKB0wCYwiv0zdNsjXFPkpqUyBTBj1DBdrRf0sxr'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

def createSet(corpus):

    DataSet = []
    for index, tweet in corpus.iterrows():

        try:
            tweetFetched = api.get_status(tweet["tweet_id"])
            print("Tweet Achado:\n" + tweetFetched.text)
            DataSet.append((tweet["tweet_id"],tweetFetched.text))

        except:
            print("Exception")
            continue

    return DataSet

resultFile = createSet(df.drop_duplicates(subset= ['tweet_id']))

Tweet Achado:
@ForeverPlayerG tem que ser , essa o áudio tava ruim . Próxima faz a farm de iron na live.
Tweet Achado:
@leozitotevez Que tristeza em que vc estava pra se lembrar disso hein
Tweet Achado:
esse João Guilherme ta mt estranho
Tweet Achado:
não quero ir pra escola
Tweet Achado:
@AbioJoao Patrão
Tweet Achado:
Preciso acordar mais cedo e ajeitar meu cabelo que tá só a bagaceira
Tweet Achado:
#depoisdojogo Piperno se o Trainee R.Ceni passar em qualquer um dos Mata a Mata, ;você vai ter que usar um crachá De Trainee dom que vem!
Tweet Achado:
Tenho Msm irmão kkkkk https://t.co/9YkN6SehqG
Tweet Achado:
Quero conversar com alguém mano
Exception
Exception
Tweet Achado:
Mas fica em paz! Eu te amo!
Tweet Achado:
Um dia tá tudo bem, no outro não e assim vai indo a vida
Tweet Achado:
quando se fala mercado sempre tem aquele amigo p te lembrar né incrivel
Tweet Achado:
Eu me atrapalhei toda hj
Tweet Achado:
Tendeu sim, não te faz kkkk
Tweet Achado:
@gustaaps Aah sim uahauahua aha
Tweet 

In [0]:
sep = "\t"
with open('/content/drive/My Drive/Colab Notebooks/tweets.txt', 'w+') as f:
    f.write("tweet_id%stweet\n" % sep)
    for x in resultFile:
        f.write("tweet_id: %s%s" % (x[0], sep))
        if x[1] is not None:
            string = x[1]

        f.write("tweet: %s\n" % string)  

In [0]:
import spacy
import csv

def converte_dataset():
  
    with open('/content/drive/My Drive/Colab Notebooks/tweets.txt', 'r') as in_file:
        nlp = spacy.load('pt')
        name = "Sentence #\tWord\tPOS\tTag\n"
        tweets = in_file.read().split("tweet_id:")
    with open('/content/drive/My Drive/Colab Notebooks/dataset_ptbr_tweets.csv', 'w+') as file:
        df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/dataset.ptbr_tweets.txt', sep="\t")
        df = df.sort_values('tweet_id')
        file.write(name)
        seq = []
        index = 1

        for i, tweet in enumerate(tweets):
            if i == 0:
                continue
            
            tweet_id, m = tweet.split("tweet: ")
            msg = m.replace("'", " ")
            msg = msg.replace('"', ' ')
            msg = msg.replace(",", " , ")
            msg = msg.replace("\n", " ")
            msg = msg.replace("\t", " ")
            msg_nlp = nlp(u"%s" % msg)
            df_tw = df.loc[df['tweet_id'] == int(tweet_id)]
            
            for token in msg_nlp:
                seq.append([
                    token.text,
                    token.pos_,
                    'O'
                ])
                
            words = [token.text for token in msg_nlp]
            
            for ind, lin in df_tw.iterrows():
                if lin['named_entity_type'] != 'No entities found in this tweet':
                    tag = lin['named_entity_type']
                    start = lin['start_pos'] -1
                    end = lin['end_pos'] -1
              
                    str1 = [
                        m[start:end], 
                        m[(start+1):(end+1)],
                        m[(start+2):(end+2)],
                    ]    
                    
                    stop = False
                    for palavra in str1:
                        for cont, word in enumerate(words):
                            if palavra == word:
                                seq[cont][2] = tag
                                stop = True
                                break
                                
                        if stop:
                            break
            
            for sent in seq:
                if sent[0] != ' ':
                    file.write("Sentence %d\t" % index)
                    file.write("%s\t" % sent[0])
                    file.write("%s\t" % sent[1])
                    file.write("%s\n" % sent[2])    
            
            index += 1   

In [0]:
converte_dataset()

In [0]:
#Reading the dataset ptbr_tweets
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset_ptbr_tweets.csv', delimiter='\t',encoding = 'utf-8')
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence 1,@ForeverPlayerG,PROPN,O
1,Sentence 1,tem,AUX,O
2,Sentence 1,que,SCONJ,O
3,Sentence 1,ser,VERB,O
4,Sentence 1,",",PUNCT,O
5,Sentence 1,essa,DET,O
6,Sentence 1,o,DET,O
7,Sentence 1,áudio,NOUN,O
8,Sentence 1,tava,VERB,O
9,Sentence 1,ruim,NOUN,O


In [0]:
df.describe()

,Sentence #,Word,POS,Tag
count,2609909,2609230,2609909,2609909
unique,670,3080,16,7
top,Sentence 670,",",NOUN,O
freq,7867,74693,466655,2595980


In [0]:
#Displaying the unique Tags
df['Tag'].unique()

array(['O', 'B-person', 'I-person', 'B-location', 'I-location',
       'B-organization', 'I-organization'], dtype=object)

In [0]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-location,2936
1,B-organization,2053
2,B-person,4797
3,I-location,2510
4,I-organization,361
5,I-person,1272
6,O,2595980


In [0]:
#Checking null values, if any.
df.isnull().sum()

Sentence #      0
Word          679
POS             0
Tag             0
dtype: int64

In [0]:
df = df.fillna(method = 'ffill')

In [0]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [0]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'@ForeverPlayerG tem que ser , essa o áudio tava ruim . Próxima faz a farm de iron na live .'

In [0]:
sentences = getter.sentences
sentences[0]

[('@ForeverPlayerG', 'PROPN', 'O'),
 ('tem', 'AUX', 'O'),
 ('que', 'SCONJ', 'O'),
 ('ser', 'VERB', 'O'),
 (',', 'PUNCT', 'O'),
 ('essa', 'DET', 'O'),
 ('o', 'DET', 'O'),
 ('áudio', 'NOUN', 'O'),
 ('tava', 'VERB', 'O'),
 ('ruim', 'NOUN', 'O'),
 ('.', 'PUNCT', 'O'),
 ('Próxima', 'PROPN', 'O'),
 ('faz', 'VERB', 'O'),
 ('a', 'ADP', 'O'),
 ('farm', 'VERB', 'O'),
 ('de', 'ADP', 'O'),
 ('iron', 'NOUN', 'O'),
 ('na', 'ADP', 'O'),
 ('live', 'NOUN', 'O'),
 ('.', 'PUNCT', 'O')]

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
          'bias': 1.0,
          'word.lower()': word.lower(),
          'hashtag':word.startswith('#') and len(word) > 1,
          'mention':word.startswith('@') and len(word) > 1,
          'word.isalnum()':word.isalnum(),
          'url':word.startswith('https://'), 
          'word[-3:]': word[-3:],
          'word[-2:]': word[-2:],
          'word.isupper()': word.isupper(),
          'word.istitle()': word.istitle(),
          'word.isdigit()': word.isdigit(),
          'postag': postag,
          'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.isalnum()':word1.isalnum(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:mention':word1.startswith('@') and len(word) > 1,
            '-1:hashtag':word1.startswith('#') and len(word) > 1,
            '-1:url':word.startswith('https://'),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+:word.isalnum()':word1.isalnum(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:mention':word1.startswith('@') and len(word) > 1,
            '+1:hashtag':word1.startswith('#') and len(word) > 1,
            '+1:url':word.startswith('https://'),
        })
    else:
        features['EOS'] = True
  
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]  

In [0]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [0]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [0]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

In [0]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.995985199816475


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
report = flat_classification_report(y_test, y_pred)
print(report)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                precision    recall  f1-score   support

    B-location       0.71      0.42      0.53       674
B-organization       0.68      0.28      0.40       320
      B-person       0.66      0.46      0.54       971
    I-location       0.82      0.36      0.50       303
I-organization       0.00      0.00      0.00        83
      I-person       0.34      0.18      0.23       259
             O       1.00      1.00      1.00    526267

      accuracy                           1.00    528877
     macro avg       0.60      0.39      0.46    528877
  weighted avg       1.00      1.00      1.00    528877

